In [1]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from PIL import Image, ExifTags
import uuid

In [2]:
"""
Es sind insgesamt 444670 Bilder, deshalb kann max_files gesetzt werden und tqdm läuft akkurat.
Eine andere Möglichkeit die tqdm bar anzupassen gibt es nicht, weil die Bilder dynamisch geladen werden.

"""

def find_image_files(root_dir, extensions=('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif'), max_files=5000):
    image_files = []
    # Initialisierung von tqdm außerhalb von os.walk, um die Anzahl der Dateien zu zählen
    pbar = tqdm(total=max_files, desc='Durchsuche Verzeichnisse')
    
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(extensions):
                image_files.append(os.path.join(subdir, file))
                pbar.update(1)
                if len(image_files) >= max_files:
                    pbar.close()
                    return image_files  # Beendet die Suche, wenn das Limit erreicht ist
    pbar.close()  # Schließe den tqdm-Balken, wenn die Suche beendet ist
    return image_files

# Verwenden der Funktion, um Bildpfade aus dem 'data'-Verzeichnis zu sammeln
image_paths = find_image_files("D:\\data\\image_data")

# Anzeigen der Anzahl der gefundenen Bilder und der ersten 10 Bildpfade zur Überprüfung
print(f"Anzahl gefundener Bilder: {len(image_paths)}")
if len(image_paths) > 10:
    print("Einige der gefundenen Bildpfade:", image_paths[:10])
else:
    print("Gefundene Bildpfade:", image_paths)

Durchsuche Verzeichnisse:   0%|          | 0/5000 [00:00<?, ?it/s]

Anzahl gefundener Bilder: 5000
Einige der gefundenen Bildpfade: ['D:\\data\\image_data\\coco2017_train\\train2017\\000000000034.jpg', 'D:\\data\\image_data\\coco2017_train\\train2017\\000000000049.jpg', 'D:\\data\\image_data\\coco2017_train\\train2017\\000000000071.jpg', 'D:\\data\\image_data\\coco2017_train\\train2017\\000000000078.jpg', 'D:\\data\\image_data\\coco2017_train\\train2017\\000000000081.jpg', 'D:\\data\\image_data\\coco2017_train\\train2017\\000000000089.jpg', 'D:\\data\\image_data\\coco2017_train\\train2017\\000000000208.jpg', 'D:\\data\\image_data\\coco2017_train\\train2017\\000000000241.jpg', 'D:\\data\\image_data\\coco2017_train\\train2017\\000000000247.jpg', 'D:\\data\\image_data\\coco2017_train\\train2017\\000000000283.jpg']


In [3]:
'''
Farbwerte werden auf RGB gesetzt.
'''
def extract_image_details(image_path):
    with Image.open(image_path) as img:
        img = img.convert('RGB')  # Konvertierung zu RGB

        
        # Farbwerte und Helligkeit
        pixels = list(img.getdata())
        avg_color = tuple(sum(col) // len(pixels) for col in zip(*pixels))
        avg_brightness = sum(sum(pixel) for pixel in pixels) // (3 * len(pixels))
        
        # Auflösung und DPI
        resolution = img.size  # (width, height)
        dpi = img.info.get('dpi', (0, 0)) 
        
        # Metadaten extrahieren
        try:
            exif_data = img._getexif()
            metadata = {ExifTags.TAGS[k]: v for k, v in exif_data.items() if k in ExifTags.TAGS} if exif_data else {}
        except AttributeError:
            metadata = {}

        # Dateidetails
        file_size = os.path.getsize(image_path)
        file_type = os.path.splitext(image_path)[1]
        
        # UUID
        unique_id = str(uuid.uuid4())
    
    return {
        'ID': unique_id,
        'File_Path': image_path,
        'Average_Color': avg_color,
        'Brightness': avg_brightness,
        'Resolution': resolution,
        'DPI': dpi,
        'File_Size': file_size,
        'File_Type': file_type,
        'Metadata': metadata
    }

In [4]:
'''
Batch size kann angepasst werden. 
Progressbar zeigt verarbeitete batches.
'''
def image_batch_generator(image_files, batch_size=5):
    total_batches = (len(image_files) + batch_size - 1) // batch_size
    progress_bar = tqdm(total=total_batches, desc="Processing images")
    
    for index in range(0, len(image_files), batch_size):
        batch = image_files[index:index + batch_size]
        details_list = [extract_image_details(image_path) for image_path in batch]
        df = pd.DataFrame(details_list)
        yield df
        progress_bar.update(1)
    
    progress_bar.close() 

In [5]:

for image_data in image_batch_generator(image_paths):
    print(image_data) 
# Hier SQLite Logik

Processing images:   0%|          | 0/1000 [00:00<?, ?it/s]

                                     ID  \
0  24e94cfb-564e-46c0-9baa-3547faee0cac   
1  986e0875-1bfe-40cd-90d4-1fc90ba6e147   
2  522c14a6-815d-42a7-8028-d57dde0c6cfb   
3  70e400f7-448b-4c3a-ba9c-0483a90625f4   
4  b8a17c4d-fc4c-4e2e-b284-b879f9b1781d   

                                           File_Path    Average_Color  \
0  D:\data\image_data\coco2017_train\train2017\00...   (109, 116, 52)   
1  D:\data\image_data\coco2017_train\train2017\00...  (135, 148, 122)   
2  D:\data\image_data\coco2017_train\train2017\00...  (101, 111, 117)   
3  D:\data\image_data\coco2017_train\train2017\00...  (177, 145, 131)   
4  D:\data\image_data\coco2017_train\train2017\00...  (154, 166, 179)   

   Brightness  Resolution         DPI  File_Size File_Type Metadata  
0          92  (640, 425)    (72, 72)     406018      .jpg       {}  
1         135  (381, 500)    (72, 72)     124619      .jpg       {}  
2         109  (640, 426)  (300, 300)     214185      .jpg       {}  
3         151  (612, 6

KeyboardInterrupt: 